<a href="https://colab.research.google.com/github/mohamedahmedsaadahmed77/Research-Project-Selected-2/blob/master/LR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%tensorflow_version 1.x
import tensorflow as tf
print(tf.__version__)

TensorFlow 1.x selected.
1.15.2


In [0]:
!git clone https://github.com/mohamedahmedsaadahmed77/OCR-Selected-2.git

Cloning into 'OCR-Selected-2'...
remote: Enumerating objects: 11, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 183726 (delta 0), reused 9 (delta 0), pack-reused 183715
Receiving objects: 100% (183726/183726), 64.07 MiB | 28.54 MiB/s, done.
Resolving deltas: 100% (142652/142652), done.
Checking out files: 100% (107101/107101), done.


In [0]:
import numpy as np
import os
from imutils import paths
import cv2
import pandas as pd

def load_images(path):
  print("[INFO] loading images...")
  imagePaths = list(paths.list_images(path))
  imagePaths.sort()
  data = []
  labels = []
  for imagePath in imagePaths:
    label = imagePath.split(os.path.sep)[-2]
    image = cv2.imread(imagePath)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image = np.reshape(image, (28*28))/255.0
    data.append(image)
    labels.append(label)
  data = np.array(data)
  labels = pd.get_dummies(labels)
  labels = np.array(labels)
  print("[INFO] done")
  return data,labels

In [0]:
x_train, y_train = load_images('/content/OCR-Selected-2/Dataset/Training')
x_test, y_test = load_images('/content/OCR-Selected-2/Dataset/Test')

[INFO] loading images...
[INFO] done
[INFO] loading images...
[INFO] done


In [0]:
from sklearn.decomposition import PCA

pca = PCA(.95)
pca.fit(x_train)

PCA(copy=True, iterated_power='auto', n_components=0.95, random_state=None,
    svd_solver='auto', tol=0.0, whiten=False)

In [0]:
x_train = pca.transform(x_train)
x_test = pca.transform(x_test)

print(x_train.shape)
print(x_test.shape)

(88799, 116)
(18299, 116)


In [0]:
# Hyper parameters
learning_rate = 0.0001
epochs = 32
batch_size = 16
batches = int(x_train.shape[0] / batch_size)


# I choose placeholder to make it recieve any number of records to make code flexable
X = tf.placeholder(tf.float32, [None, 116])
Y = tf.placeholder(tf.float32, [None, 26])

# I choose variable cause it’s values will be changed to get more better values and i put random values for the first time
W = tf.Variable(.1 * np.random.randn(116, 26).astype(np.float32))
B = tf.Variable(.1 * np.random.randn(26).astype(np.float32))

In [0]:
# Formula of logistic regression : X * W + B
pred = tf.nn.softmax(tf.add(tf.matmul(X,W), B))

# Cost function
cost = tf.reduce_mean(-tf.reduce_sum(Y * tf.log(pred), axis=1))

# Optimiser that will get better values for bais and weights
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

In [0]:
# Session to determine the flow of computitional graph (TensorFlow)
with tf.Session() as sesh:
    # Set initial values of tensor variables (Mandatory)
    sesh.run(tf.global_variables_initializer())
    
    for epoch in range(epochs):
        for i in range(batches):
            offset = i * epoch
            x = x_train[offset: offset + batch_size]
            y = y_train[offset: offset + batch_size]
            sesh.run(optimizer, feed_dict={X: x, Y:y})
            # Get cost function value (Optional)
            costVal = sesh.run(cost, feed_dict={X:x, Y:y})
        print(f'epoch: {epoch:2d} cost_val= {costVal:.4f}')
    
    # Calculate the accuracy
    correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(Y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
    acc = accuracy.eval({X: x_test, Y: y_test})
    print(f'Accuracy: {acc * 100:.2f}%')

epoch:  0 cost_val= 0.4302
epoch:  1 cost_val= 1.8001
epoch:  2 cost_val= 3.4093
epoch:  3 cost_val= 2.2630
epoch:  4 cost_val= 2.4115
epoch:  5 cost_val= 4.1587
epoch:  6 cost_val= 4.4491
epoch:  7 cost_val= 3.5939
epoch:  8 cost_val= 4.0873
epoch:  9 cost_val= 3.5260
epoch: 10 cost_val= 3.6516
epoch: 11 cost_val= 3.5959
epoch: 12 cost_val= 3.7382
epoch: 13 cost_val= 4.3422
epoch: 14 cost_val= 4.2943
epoch: 15 cost_val= 3.7203
epoch: 16 cost_val= 3.8531
epoch: 17 cost_val= nan
epoch: 18 cost_val= nan
epoch: 19 cost_val= nan
epoch: 20 cost_val= nan
epoch: 21 cost_val= nan
epoch: 22 cost_val= nan
epoch: 23 cost_val= nan
epoch: 24 cost_val= nan
epoch: 25 cost_val= nan
epoch: 26 cost_val= nan
epoch: 27 cost_val= nan
epoch: 28 cost_val= nan
epoch: 29 cost_val= nan
epoch: 30 cost_val= nan
epoch: 31 cost_val= nan
Accuracy: 54.28%
